<h2>Patients and Doctors Representations</h2>

In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install transformers

In [3]:
import numpy as np
import pandas as pd

In [4]:
patients = pd.read_csv('../Datasets/patients_data_final.csv')
patients.head(5)

,UserId,Age,Gender,MedicalHistory
0,1,42,Male,"[{'Disease': 'Tuberculosis', 'Symptoms': ""[' h..."
1,2,35,Female,"[{'Disease': 'Common Cold', 'Symptoms': ""[' hi..."
2,3,45,Female,"[{'Disease': 'Common Cold', 'Symptoms': ""[' hi..."
3,4,39,Female,"[{'Disease': 'Common Cold', 'Symptoms': ""[' lo..."
4,5,37,Male,"[{'Disease': 'Common Cold', 'Symptoms': ""[' lo..."


<h3>Patients Representation using Bio-BERT Model</h3>

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load BioBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")

# Load BioBERT model
model = AutoModel.from_pretrained("monologg/biobert_v1.1_pubmed")

In [6]:
def encode(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the pooled output (representation of the entire input sequence)
    pooled_output = outputs.last_hidden_state.mean(dim=1)

    # Convert to numpy array
    pooled_output = pooled_output.numpy()

    return pooled_output

In [21]:
import ast

patients_list = []

# num_rows = patients.shape[0]
num_rows = 1000
for index in range (0, num_rows):
    text = ""
    medhis = patients.iloc[index]['MedicalHistory']
    medhis = ast.literal_eval(medhis)
    for his in medhis:
        dis = his['Disease']
        text += dis
        symp = his['Symptoms']
        symp = ast.literal_eval(symp)
        for sy in symp:
            text += sy
        text += " "

    embed = encode(text)
    # print(embed.shape)
    embed_final = embed.reshape(-1)
    # print(type(embed_final))
    cur_age = (patients.iloc[index]['Age'] - 1) / 99
    cur_gender = patients.iloc[index]['Gender']
    fin_gender = 1
    if(cur_gender == "Male"):
        fin_gender = 0

    # Convert cur_age to a 1D array
    cur_age_array = np.array([cur_age])

    # Convert fin_gender to a 1D array
    fin_gender_array = np.array([fin_gender])

    # Concatenate cur_age_array and fin_gender_array with embed_final along axis 1
    embed_final = np.concatenate((cur_age_array, fin_gender_array, embed_final), axis=0)

    patients_list.append(embed_final)
 
patients_rep = pd.DataFrame(patients_list)

In [22]:
patients_rep.head(5)

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,768,769
0,0.414141,0.0,0.128810,-0.016112,-0.187876,-0.034305,-0.252311,0.169892,0.063047,-0.070421,...,-0.103592,0.038848,0.019238,-0.319696,-0.035312,0.094906,-0.107771,0.047160,0.082996,0.174099
1,0.343434,1.0,0.091672,-0.259791,-0.035402,0.085763,-0.390162,0.145556,0.096970,-0.007537,...,-0.205160,0.146655,-0.053556,-0.498021,-0.134685,-0.120590,0.066527,0.263349,-0.014404,0.077126
2,0.444444,1.0,0.076476,-0.171781,0.058706,-0.014736,-0.518227,0.193263,-0.056187,0.004813,...,-0.162047,0.212863,-0.297990,-0.372694,-0.254428,-0.124167,-0.089783,0.249434,0.068945,0.262406
3,0.383838,1.0,0.129531,0.025772,-0.123044,-0.163594,-0.332583,0.103516,-0.002976,-0.073760,...,-0.063982,0.083277,-0.150059,-0.282145,-0.132049,0.081419,-0.019150,-0.067709,0.170656,0.202101
4,0.363636,0.0,0.200941,-0.012209,-0.031935,-0.024259,-0.403893,0.073753,0.060491,-0.053991,...,-0.061866,-0.014798,-0.192443,-0.236006,-0.309018,-0.006340,0.022623,0.013366,0.119302,0.335641


In [23]:
patients_rep.to_csv('../Datasets/patients_representation.csv', index=False)

<h3>Doctors Representation</h3>

In [12]:
doctors = pd.read_csv('../Datasets/doctors_data_multireward.csv')
doctors.head(5)

,Doctor Speciality,Doctor Name,Ratings,Experience,Distance from Patient,Availability,Cost of Services
0,Dermatologist,Dr. Kristin Butler,3.81,1.61,8.77,1,19.74
1,Dermatologist,Dr. Brett Diaz,3.47,1.83,7.91,1,1.78
2,Dermatologist,Dr. Michael Maxwell,4.56,3.68,7.37,1,4.05
3,Dermatologist,Dr. Ronald Bullock,3.20,19.54,15.44,1,6.83
4,Dermatologist,Dr. Donna Brown,4.14,0.81,3.17,1,2.85
